In [1]:
import geopandas as gpd
from shapely import ops
from functools import partial

In [2]:
input_file = "2017_PHC_Fiji_Tikina_4326.geojson"
output_file = "2017_PHC_Fiji_Tikina_4326_fixed.geojson"

In [3]:
def normalize_lon(lon):
    """Normalize longitude to -180 to 180"""
    return ((lon + 180) % 360 - 180)


def shift_geometry(geom):
    """Recursively shift coordinates of any geometry type"""

    def _shift_coords(x, y, z=None):
        x = normalize_lon(x)
        if z is not None:
            return (x, y, z)
        else:
            return (x, y)

    return ops.transform(_shift_coords, geom)

In [4]:
def shift_lon(lon):
    """Convert -180..180 to 0..360 format"""
    return lon % 360


def shift_geometry_positive(geom):
    def _shift_coords(x, y, z=None):
        x = shift_lon(x)
        if z is not None:
            return (x, y, z)
        else:
            return (x, y)

    return ops.transform(_shift_coords, geom)

In [5]:
gdf = gpd.read_file(input_file)

In [6]:
# Reproject to WGS84 (EPSG:4326) just in case
if gdf.crs != "EPSG:4326":
    print("Reprojecting to EPSG:4326...")
    gdf = gdf.to_crs("EPSG:4326")

# Fix longitudes across International Date Line
print("Normalizing longitudes...")
gdf["geometry"] = gdf["geometry"].apply(shift_geometry)
gdf["geometry"] = gdf["geometry"].apply(shift_geometry_positive)
gdf = gdf[gdf.is_valid]

print(f"Saving fixed GeoJSON to {output_file}...")
gdf.to_file(output_file, driver="GeoJSON")

Normalizing longitudes...


/home/dedenbangkit/.local/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/home/dedenbangkit/.local/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


Saving fixed GeoJSON to 2017_PHC_Fiji_Tikina_4326_fixed.geojson...
